<a href="https://colab.research.google.com/github/vijayaahirejadhav/DataScience/blob/main/Mercedes_Benz_Greener_Manufacturing_VSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="http://cfs22.simplicdn.net/ice9/new_logo.svgz"/>

# Mercedes-Benz Greener Manufacturing

### DESCRIPTION

##### Background of Problem Statement :

Project 1 - Mercedes-Benz Greener Manufacturing

DESCRIPTION

Reduce the time a Mercedes-Benz spends on the test bench.

# Problem Statement Scenario:
Since the first automobile, the Benz Patent Motor Car in 1886, Mercedes-Benz has stood for important automotive innovations. These include the passenger safety cell with the crumple zone, the airbag, and intelligent assistance systems. Mercedes-Benz applies for nearly 2000 patents per year, making the brand the European leader among premium carmakers. Mercedes-Benz cars are leaders in the premium car industry. With a huge selection of features and options, customers can choose the customized Mercedes-Benz of their dreams.

To ensure the safety and reliability of every unique car configuration before they hit the road, Daimler’s engineers have developed a robust testing system. As one of the world’s biggest manufacturers of premium cars, safety and efficiency are paramount on Daimler’s production lines. However, optimizing the speed of their testing system for many possible feature combinations is complex and time-consuming without a powerful algorithmic approach.

You are required to reduce the time that cars spend on the test bench. Others will work with a dataset representing different permutations of features in a Mercedes-Benz car to predict the time it takes to pass testing. Optimal algorithms will contribute to faster testing, resulting in lower carbon dioxide emissions without reducing Daimler’s standards.

# Following actions should be performed:
* If for any column(s), the variance is equal to zero, then you need to remove those variable(s).
* Check for null and unique values for test and train sets
* Apply label encoder.
* Perform dimensionality reduction.
* Predict your test_df values using xgboost


**Happy coding!**

* * *

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.impute import SimpleImputer

# Load data here

In [ ]:
#path=r'data\train.csv'
#df= pd.read_csv(path)
df_train=pd.read_csv("./data/train.csv")
df_test=pd.read_csv("./data/test.csv")

In [ ]:
df_train.head(5)
print(df_train.shape)

(4209, 378)


In [ ]:
df_test.head(5)
print(df_test.shape)

(4209, 377)


In [ ]:
# seperate the y from the data as we will use this to learn as 
# the prediction output
y_train = df_train['y'].values
df_train=df_train.drop(['ID','y'],axis=1)
x_train = df_train
x_test = df_test.drop(['ID'],axis=1)


# If for any column(s), the variance is equal to zero, then you need to remove those variable(s).

In [ ]:
# Check for null values
def check_null_missing_values(f):
    if f.isnull().any().any():
        print("Found missing values in dataframe")
    else:
        print("No missing values in dataframe")

check_null_missing_values(x_train)
check_null_missing_values(x_test)

No missing values in dataframe
No missing values in dataframe


In [ ]:
print(x_train.shape)
print(x_test.shape)

(4209, 376)
(4209, 376)


In [ ]:
#Convert columns into list
col=x_train.columns.values.tolist()

In [ ]:
drop_column=[]#Collect unique values from dataframe to remove
for c in col:
    count = len(np.unique(x_train[c]))
    if count == 1:
        drop_column.append(c)
# Drop the unique value columns from train and test dataset
x_test.drop(drop_column,axis=1,inplace=True)
x_train.drop(drop_column,axis=1,inplace=True)

In [ ]:
print(x_train.shape)
print(x_test.shape)

(4209, 364)
(4209, 364)


# Apply label encoder.

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb_enc = LabelEncoder()

In [ ]:
X_train=x_train.apply(LabelEncoder().fit_transform)

In [ ]:
X_train.head(5)

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X12,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,32,23,17,0,3,24,9,14,0,0,...,0,0,1,0,0,0,0,0,0,0
1,32,21,19,4,3,28,11,14,0,0,...,1,0,0,0,0,0,0,0,0,0
2,20,24,34,2,3,27,9,23,0,0,...,0,0,0,0,0,0,1,0,0,0
3,20,21,34,5,3,27,11,4,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20,23,34,5,3,12,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_test=x_test.apply(LabelEncoder().fit_transform)

In [ ]:
X_test.head(5)

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X12,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,21,23,34,5,3,26,0,22,0,0,...,0,0,0,1,0,0,0,0,0,0
1,42,3,8,0,3,9,6,24,0,0,...,0,0,1,0,0,0,0,0,0,0
2,21,23,17,5,3,0,9,9,0,0,...,0,0,0,1,0,0,0,0,0,0
3,21,13,34,5,3,31,11,13,0,0,...,0,0,0,1,0,0,0,0,0,0
4,45,20,17,2,3,30,8,12,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
X_test.shape

(4209, 364)

In [ ]:
X_train.shape

(4209, 364)

# Perform dimensionality reduction
### Linear dimensionality reduction using Singular Value Decomposition of the data to project it to a lower dimensional space.


In [ ]:
from sklearn.decomposition import PCA

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=True, fit_path=Tru

In [ ]:
pca = PCA(n_components=0.95)
pca_results_train = pca.fit_transform(X_train)
pca_results_test = pca.transform(X_test)

In [ ]:
pca_results_train.shape

(4209, 6)

In [ ]:
pca_results_test.shape

(4209, 6)

# Predict your test_df values using xgboost

In [ ]:
## Model Libraries
from xgboost import XGBClassifier
from xgboost import XGBRegressor

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest=train_test_split(pca_results_train, y_train, test_size=0.15)

In [ ]:
print("Xtrain Shape",xtrain.shape)
print("Ytrain Shape",ytrain.shape)
print("Xtest Shape",xtest.shape)
print("Ytest Shape",ytest.shape)

Xtrain Shape (3577, 6)
Ytrain Shape (3577,)
Xtest Shape (632, 6)
Ytest Shape (632,)


In [ ]:
# Fit the model
model=XGBRegressor()

In [ ]:
#training the model
model.fit(xtrain,ytrain)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=100, n_jobs=4, num_parallel_tree=1,
       objective='reg:squarederror', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
       validate_parameters=1, verbosity=None)

In [ ]:
#calculating score on training data
model.score(xtrain,ytrain)

0.8593934381990184

In [ ]:
#training the test model
model.fit(xtest,ytest)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=100, n_jobs=4, num_parallel_tree=1,
       objective='reg:squarederror', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
       validate_parameters=1, verbosity=None)

In [ ]:
model.score(xtest,ytest)

0.9982650613984976